<a href="https://colab.research.google.com/github/adautofbn/ri_labs/blob/master/lab06/modelo_vetorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [358]:
import pandas as pd
import numpy as np
import nltk
import re
import collections
import bisect
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
result = pd.read_csv('https://raw.githubusercontent.com/adautofbn/ri_labs/master/lab06/results.csv')
                                            # Resultados adquiridos do site brasil-247 em abril de 2019

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade. Se você já estiver usando esses dados não precisa reconstruir o índice;

In [0]:
tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indexes = {}
n = 0

for text in result.text:
  words = [word for word in tknz.tokenize(text.lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  n += 1
  for t in words:
    if t not in indexes.keys():
      indexes[t] = []
    indexes[t].append(n)
    
for elem in indexes.items():
  d = dict(collections.Counter(elem[1]))
  indexes[elem[0]] = list(d.items())

In [360]:
indexes_df = pd.DataFrame()
indexes_df['Word'] = indexes.keys()
indexes_df['Documents and Frequencies'] = indexes.values()

indexes_df.head(10)

,Word,Documents and Frequencies
0,juíza,"[(1, 2), (2, 1)]"
1,federal,"[(1, 2), (2, 2), (3, 1), (6, 1), (7, 3), (15, ..."
2,ivani,"[(1, 1), (2, 1)]"
3,silva,"[(1, 3), (2, 1), (6, 1), (14, 2), (26, 1), (73..."
4,luz,"[(1, 3), (2, 1), (9, 1), (17, 1), (32, 2), (78..."
5,brasília,"[(1, 1), (8, 1), (33, 1), (35, 1), (44, 1), (4..."
6,proibiu,"[(1, 1), (2, 1), (119, 1), (162, 1)]"
7,caráter,"[(1, 1), (15, 1), (36, 1), (60, 1), (89, 1), (..."
8,liminar,"[(1, 1), (2, 3), (119, 1), (217, 1)]"
9,nesta,"[(1, 2), (3, 1), (4, 1), (8, 1), (21, 1), (22,..."


## 2. Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário; 

In [0]:
M = result.text.count()
for word in indexes:
  k = len(indexes[word])
  IDF = round(np.log((M+1)/k),2)
  indexes[word].append(IDF)

In [362]:
indexes_df['IDF'] = [index[-1] for index in indexes.values()]

indexes_df.head(10)

,Word,Documents and Frequencies,IDF
0,juíza,"[(1, 2), (2, 1), 4.83]",4.83
1,federal,"[(1, 2), (2, 2), (3, 1), (6, 1), (7, 3), (15, ...",1.63
2,ivani,"[(1, 1), (2, 1), 4.83]",4.83
3,silva,"[(1, 3), (2, 1), (6, 1), (14, 2), (26, 1), (73...",3.04
4,luz,"[(1, 3), (2, 1), (9, 1), (17, 1), (32, 2), (78...",2.53
5,brasília,"[(1, 1), (8, 1), (33, 1), (35, 1), (44, 1), (4...",2.19
6,proibiu,"[(1, 1), (2, 1), (119, 1), (162, 1), 4.14]",4.14
7,caráter,"[(1, 1), (15, 1), (36, 1), (60, 1), (89, 1), (...",3.22
8,liminar,"[(1, 1), (2, 3), (119, 1), (217, 1), 4.14]",4.14
9,nesta,"[(1, 2), (3, 1), (4, 1), (8, 1), (21, 1), (22,...",0.91


## 3. Implementar as seguintes versões do modelo vetorial:

### 3.1. Representação binária

In [0]:
def binary_vsm(query, document):
  score = 0
  query_tokens = query.split()
  doc_tokens = document.split()
  
  for token in query_tokens:
    score += (token in doc_tokens)
    
  return score

### 3.2. TF (lembre-se que esse modelo já está implementado)

In [0]:
def tf_vsm(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    score += doc_tokens.count(word)
  
  return score

### 3.3 TF-IDF

In [0]:
def tfidf_vsm(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    cwd = doc_tokens.count(word)
    if word in indexes:
      score += cwd * indexes[word][-1]
  
  return round(score,2)

### 3.4 BM25 (não usaremos Okapi já que os documentos não tem grande variação de tamanho)

In [0]:
def bm25_vsm(query, document, k):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  words = [word for word in query_tokens if word in doc_tokens]
    
  for word in words:
    cwd = doc_tokens.count(word)
    dfw = 0
    if word in indexes:
      dfw = len(indexes[word][:-1])
    score += (((k+1) * cwd) / (cwd + k)) * np.log10(((M+1) / dfw)) if dfw != 0 else 0
  
  return round(score,2)

## 4. Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta;

In [0]:
queries = ['jair bolsonaro', 'futebol brasileiro', 'ministro da educação']

In [0]:
def create_top5_models(query):
  n = 0
  db = []
  dtf = []
  dtfidf = []
  dbm25 = []
  for doc in result.text:
    doc = doc.lower()
    n += 1
    bisect.insort(db, (binary_vsm(query, doc), n))
    bisect.insort(dtf, (tf_vsm(query,doc), n))
    bisect.insort(dtfidf, (tfidf_vsm(query,doc), n))
    bisect.insort(dbm25, (bm25_vsm(query,doc,20), n))
  
  db.reverse()
  dtf.reverse()
  dtfidf.reverse()
  dbm25.reverse()
  
  return db[:5], dtf[:5], dtfidf[:5], dbm25[:5]

In [0]:
top5_binary = ['','','']
top5_tf = ['','','']
top5_tfidf = ['','','']
top5_bm25 = ['','','']

top5_binary[0], top5_tf[0], top5_tfidf[0], top5_bm25[0] = create_top5_models(queries[0])
top5_binary[1], top5_tf[1], top5_tfidf[1], top5_bm25[1] = create_top5_models(queries[1])
top5_binary[2], top5_tf[2], top5_tfidf[2], top5_bm25[2] = create_top5_models(queries[2])

In [370]:
query_df = pd.DataFrame()

query_df['Query'] = queries
query_df['Binary'] = top5_binary
query_df['TF'] = top5_tf
query_df['TF-IDF'] = top5_tfidf
query_df['BM25'] = top5_bm25

query_df.index+=1
query_df

,Query,Binary,TF,TF-IDF,BM25
1,jair bolsonaro,"[(2, 238), (2, 237), (2, 231), (2, 224), (2, 2...","[(41, 207), (37, 151), (31, 166), (19, 19), (1...","[(67.98, 207), (55.14, 151), (42.18, 166), (25...","[(15.55, 207), (12.09, 151), (8.32, 166), (5.8..."
2,futebol brasileiro,"[(2, 121), (2, 115), (2, 76), (2, 58), (2, 55)]","[(10, 115), (7, 242), (7, 47), (6, 55), (5, 118)]","[(20.92, 115), (17.36, 242), (17.36, 47), (12....","[(7.51, 115), (5.86, 242), (5.86, 47), (4.75, ..."
3,ministro da educação,"[(3, 240), (3, 233), (3, 222), (3, 221), (3, 2...","[(52, 151), (50, 115), (43, 166), (41, 221), (...","[(38.41, 221), (32.81, 222), (17.1, 215), (13....","[(10.68, 221), (10.22, 222), (6.26, 215), (5.1..."


## 5. Compare os resultados encontrados e responda.

### 5.1. Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta.

In [0]:
def get_top1_score(top5):
  return [top[0][0] for top in top5]

def get_top1_doc(top5):
  return [top[0][1] for top in top5]

def get_doc_title(docs):
  return [result.title[doc] for doc in docs]

In [0]:
score_top5_binary = get_top1_score(top5_binary)
score_top5_tf = get_top1_score(top5_tf)
score_top5_tfidf = get_top1_score(top5_tfidf)
score_top5_bm25 = get_top1_score(top5_bm25)

doc_top5_binary = get_top1_doc(top5_binary)
doc_top5_tf = get_top1_doc(top5_tf)
doc_top5_tfidf = get_top1_doc(top5_tfidf)
doc_top5_bm25 = get_top1_doc(top5_bm25)

titles_binary = get_doc_title(doc_top5_binary)
titles_tf = get_doc_title(doc_top5_tf)
titles_tfidf = get_doc_title(doc_top5_tfidf)
titles_bm25 = get_doc_title(doc_top5_bm25)

#### Representação Binária

In [373]:
binary_res = pd.DataFrame()
binary_res['Query'] = queries
binary_res['Document'] = doc_top5_binary
binary_res['Document Title'] = titles_binary
binary_res['Binary Score'] = score_top5_binary

binary_res.index += 1
binary_res

,Query,Document,Document Title,Binary Score
1,jair bolsonaro,238,Saúde mental dos estudantes mais um desafio p...,2
2,futebol brasileiro,121,“Bonucci diz o que muita gente pensa: os negro...,2
3,ministro da educação,240,Rota mata 11 pessoas em Guararema na terceira...,3


#### TF

In [374]:
tf_res = pd.DataFrame()
tf_res['Query'] = queries
tf_res['Document'] = doc_top5_tf
tf_res['Document Title'] = titles_tf
tf_res['TF Score'] = score_top5_tf

tf_res.index += 1
tf_res

,Query,Document,Document Title,TF Score
1,jair bolsonaro,207,Bolsonaro escancara cadáver insepulto da ditad...,41
2,futebol brasileiro,115,Brasil leva susto mas vence a República Tchec...,10
3,ministro da educação,151,Socialismo ‘millennial’ nos EUA,52


#### TF-IDF

In [375]:
tfidf_res = pd.DataFrame()
tfidf_res['Query'] = queries
tfidf_res['Document'] = doc_top5_tfidf
tfidf_res['Document Title'] = titles_tfidf
tfidf_res['TFIDF Score'] = score_top5_tfidf

tfidf_res.index += 1
tfidf_res

,Query,Document,Document Title,TFIDF Score
1,jair bolsonaro,207,Bolsonaro escancara cadáver insepulto da ditad...,67.98
2,futebol brasileiro,115,Brasil leva susto mas vence a República Tchec...,20.92
3,ministro da educação,221,A jovem deputada que jogou contra as cordas o ...,38.41


#### BM-25

In [376]:
bm25_res = pd.DataFrame()
bm25_res['Query'] = queries
bm25_res['Document'] = doc_top5_bm25
bm25_res['Document Title'] = titles_bm25
bm25_res['BM25 Score'] = score_top5_bm25

bm25_res.index += 1
bm25_res

,Query,Document,Document Title,BM25 Score
1,jair bolsonaro,207,Bolsonaro escancara cadáver insepulto da ditad...,15.55
2,futebol brasileiro,115,Brasil leva susto mas vence a República Tchec...,7.51
3,ministro da educação,221,A jovem deputada que jogou contra as cordas o ...,10.68


O comparativo acima mostra os títulos de cada documento que apareceu como melhor resultado para a consulta. Nesses testes podemos perceber que o TF-IDF e o BM25 apresentam resultados semelhantes e também os melhores resultados.

Para a consulta, Jair Bolsonaro, temos:
- 'Bolsonaro escancara cadáver insepulto da ditadura com celebração do golpe'

Para Futebol Brasileiro, temos:
-  'Brasil leva susto  mas vence a República Tcheca de virada'

Para Ministro da Educação, temos:
- 'A jovem deputada que jogou contra as cordas o ministro da Educação e sua “lista de desejos'

Porém, ao meu ver, pelo grau de complexidade da implementação do algoritmo, o TF-IDF apesar de simples, já nos provê um resultado satisfatório.



### 5.2. Calcule e reporte o overlap par-a-par entre os resultados de cada modelo (usando o índice de Jaccard)